In [1]:
import json

import numpy as np
import torch
import torch.nn as nn

from collections import defaultdict

from torch.utils import data
from trajdata import AgentBatch, AgentType, UnifiedDataset
from trajdata.augmentation import NoiseHistories

from modules.model_registrar import ModelRegistrar
from modules.snce import SocialNCE

In [2]:
with open('../config/pedestrians.json', 'r', encoding="utf-8") as config_json:
    hyperparams = json.load(config_json)
hyperparams["device"] = "cpu"
hyperparams["edge_encoding"] = True
hyperparams["learning_rate"] = 0.016
hyperparams["contrastive_weight"] = 50.0
hyperparams["history_sec"] = 2.8
hyperparams["prediction_sec"] = 4.8
hyperparams["incl_robot_node"] = False
hyperparams["map_encoding"] = False
hyperparams["preprocess_workers"] = 16
hyperparams["trajdata_cache_dir"] = "../data/pedestrian_datasets/.unified_data_cache"

# hyperparams

In [3]:
desired_data=[
    "eupeds_eth-train",
]
data_dirs = {
    "eupeds_eth": "~/Projects/trajectron/datasets/eth_ucy_peds",
}

attention_radius = defaultdict(
    lambda: 20.0
)  # Default range is 20m unless otherwise specified.
attention_radius[(AgentType.PEDESTRIAN, AgentType.PEDESTRIAN)] = 5.0

input_noise = 0.0
augmentations = list()
if input_noise > 0.0:
    augmentations.append(NoiseHistories(stddev=input_noise))

In [4]:
batch_size = 4

dataset = UnifiedDataset(
    desired_data=desired_data,
    history_sec=(0.1, hyperparams["history_sec"]),
    future_sec=(0.1, hyperparams["prediction_sec"]),
    agent_interaction_distances=attention_radius,
    incl_robot_future=hyperparams["incl_robot_node"],
    incl_raster_map=hyperparams["map_encoding"],
    only_predict=[AgentType.PEDESTRIAN],
    no_types=[AgentType.UNKNOWN],
    augmentations=augmentations if len(augmentations) > 0 else None,
    num_workers=hyperparams["preprocess_workers"],
    cache_location=hyperparams["trajdata_cache_dir"],
    data_dirs=data_dirs,
    verbose=True,
)

dataloader = data.DataLoader(
    dataset,
    collate_fn=dataset.get_collate_fn(pad_format="right"),
    pin_memory=False if hyperparams["device"] == "cpu" else True,
    batch_size=batch_size,
    shuffle=True,
    num_workers=hyperparams["preprocess_workers"],
    sampler=None,
)

batch: AgentBatch = next(iter(dataloader))

Loading data for matched scene tags: ['zurich-eupeds_eth-train', 'eupeds_eth-cyprus-train']


Calculating Agent Data (Serially): 100%|██████████| 1/1 [00:00<00:00, 8208.03it/s]

1 scenes in the scene index.



Structuring Agent Data Index: 100%|██████████| 1/1 [00:00<00:00, 2275.80it/s]


In [5]:
print(f"Agent Future Length: {batch.agent_fut_len}")
print(f"Neigbors Future Length: {batch.neigh_fut_len}")

Agent Future Length: tensor([ 7,  4, 12,  4])
Neigbors Future Length: tensor([[ 7,  7,  7,  7,  7,  7,  7,  7,  7],
        [ 6,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 9, 12, 11, 10,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])


In [6]:
model_dir = "./"
model_registrar = ModelRegistrar(model_dir, hyperparams["device"])

In [7]:
snce = SocialNCE(
    feat_dim=2*hyperparams["enc_rnn_dim_history"],
    proj_head_dim=hyperparams["head_proj_dim"],
    event_enc_dim=hyperparams["event_enc_dim"],
    snce_head_dim=hyperparams["snce_head_dim"],
    device=hyperparams["device"]
).to(hyperparams["device"])

In [8]:
x_size = 64
enc = torch.rand((batch_size, x_size))

In [9]:
loss_snce = snce.loss(enc, batch)
loss_snce

tensor(1.5972, grad_fn=<DivBackward0>)